In [1]:
using Flux, Statistics
using Flux.Data: DataLoader
using Flux: onehotbatch, onecold, @epochs
using Flux.Losses: logitcrossentropy
using Base: @kwdef
using MLDatasets

In [2]:
include("network.jl")

In [3]:
model = Chain(PhasorDense(1024 => 32), )

Chain(
  PhasorDense(1024 => 32),              # 32_800 parameters
) 

In [4]:
x = rand(Float32, (100,1024))

100×1024 Matrix{Float32}:
 0.925288   0.108457  0.341305    …  0.853123   0.820373    0.17382
 0.504352   0.98407   0.094907       0.453549   0.725726    0.965713
 0.759225   0.139443  0.435339       0.42683    0.520653    0.340711
 0.130162   0.656978  0.47056        0.0442799  0.569047    0.994109
 0.548466   0.27578   0.459192       0.684469   0.00468349  0.116094
 0.0923626  0.794069  0.232205    …  0.979701   0.392621    0.779744
 0.554211   0.264362  0.880811       0.159541   0.0383127   0.589179
 0.731584   0.865006  0.631056       0.348675   0.351941    0.812073
 0.433085   0.531436  0.816776       0.0504389  0.650845    0.922108
 0.929529   0.613534  0.569513       0.337689   0.740842    0.747886
 ⋮                                ⋱                         
 0.734427   0.76371   0.217499       0.443477   0.574695    0.811415
 0.50772    0.279193  0.531405       0.749589   0.610571    0.556621
 0.963155   0.144494  0.138454       0.875519   0.383271    0.847026
 0.186271   0.644

In [6]:
y = model(x)

32×100 Matrix{Float32}:
  0.155844    0.460217     0.609756    0.105603   …   0.193041    0.380222
  0.13067     0.308425     0.307364    0.359068       0.0420887   0.206883
 -0.0984334  -0.0242903   -0.0783415  -0.0675861     -0.447385   -0.0389762
  0.260787    0.0565158    0.251534    0.0671619      0.434234    0.390445
 -0.119823   -0.305864    -0.252228   -0.477746      -0.154085   -0.410489
 -0.0277456  -0.0224878    0.967001   -0.0543603  …  -0.0512968  -0.110567
 -0.149122   -0.289121    -0.318532   -0.291899      -0.487773   -0.182133
  0.0150925   0.0544917   -0.968777   -0.11531       -0.691972   -0.0832553
  0.0291548  -0.00322209  -0.0323095  -0.095149       0.143281   -0.00989137
 -0.152712   -0.127306    -0.358995   -0.430293      -0.595595   -0.249941
  ⋮                                               ⋱              
 -0.0239342  -0.485031    -0.301891   -0.0265877     -0.609      -0.936667
 -0.368672   -0.140288     0.589247   -0.189201      -0.311254   -0.0609867
  0.5

In [11]:
minimum(y)

-0.99170715f0

In [13]:
mean(y)

-0.035180368f0

In [12]:
maximum(y)

0.99845666f0

In [2]:
function getdata(args)
    ENV["DATADEPS_ALWAYS_ACCEPT"] = "true"

    @info "Getting and transforming data"

    ## Load dataset
    xtrain, ytrain = MLDatasets.MNIST(:train)[:]
    xtest, ytest = MLDatasets.MNIST(:test)[:]

    ## Reshape input data to flatten each image into a linear array
    xtrain = Flux.flatten(xtrain)
    xtest = Flux.flatten(xtest)

    ## One-hot-encode the labels
    ytrain, ytest = onehotbatch(ytrain, 0:9), onehotbatch(ytest, 0:9)

    ## Create two DataLoader objects (mini-batch iterators)
    train_loader = DataLoader((xtrain, ytrain), batchsize=args.batchsize, shuffle=true)
    test_loader = DataLoader((xtest, ytest), batchsize=args.batchsize)

    return train_loader, test_loader
end

getdata (generic function with 1 method)

In [ ]:
layer = 

In [ ]:
function build_model(; imgsize=(28,28,1), nclasses=10)
    return Chain( Dense(prod(imgsize), 32, relu),
                  Dense(32, nclasses))
end